In [1]:
import os
import codecs
import re

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

from xclib.data import data_utils #https://github.com/kunaldahiya/pyxclib

In [2]:
import os
os.chdir("/Users/sachitnagpal/Library/Mobile Documents/com~apple~CloudDocs/nyu/thesis/code/LdSM-WS/preprocess_data")

In [3]:
# exc_labels = data_utils.read_sparse_file("thesis/code/swiftxml/Tree_Extreme_Classifiers/Sandbox/Results/EUR-Lex/exc_tst_X_Y.txt").toarray()
# inc_labels = data_utils.read_sparse_file("thesis/code/swiftxml/Tree_Extreme_Classifiers/Sandbox/Results/EUR-Lex/inc_tst_X_Y.txt").toarray()
# labels = data_utils.read_sparse_file("/Users/sachitnagpal/Library/Mobile Documents/com~apple~CloudDocs/nyu/thesis/code/swiftxml/Tree_Extreme_Classifiers/Sandbox/Data/EUR-Lex/tst_X_Y.txt")

In [5]:
raw_data_path = "raw_data/AmazonCat-13K.bow"

dataset = "amazonCat13"
data_path = f"data/{dataset}/"
warm_start_data_path = f"warm_start_data/{dataset}"

In [18]:
tr_features, tr_labels, _, _, _ = data_utils.read_data(f'{raw_data_path}/train.txt')
te_features, te_labels, _, _, _ = data_utils.read_data(f'{raw_data_path}/test.txt')

In [19]:
print(tr_features.shape[0])
print(te_features.shape[0])

1186239
306782


In [7]:
tr_filter_ixs = np.array((tr_labels.sum(axis=1)>0)&(tr_features.sum(axis=1)>0)).flatten()
tr_features = tr_features[tr_filter_ixs, :]
tr_labels = tr_labels[tr_filter_ixs, :]

te_filter_ixs = np.array((te_labels.sum(axis=1)>0)&(te_features.sum(axis=1)>0)).flatten()
te_features = te_features[te_filter_ixs, :]
te_labels = te_labels[te_filter_ixs, :]

In [8]:
np.array((te_labels.sum(axis=1)<=1)).flatten().sum()

14838

In [9]:
print(len(tr_filter_ixs))
print(len(te_filter_ixs))

1186239
306782


In [11]:
# data_utils.write_data(f"{data_path}/train.txt", tr_features, tr_labels)
# data_utils.write_data(f"{data_path}/test.txt", te_features, te_labels)

In [12]:
# data_utils.write_sparse_file(tr_features, "data/amazonCat13/train_X.txt")
# data_utils.write_sparse_file(tr_labels, "data/amazonCat13/train_Y.txt")

# data_utils.write_sparse_file(te_features, "data/amazonCat13/test_X.txt")
# data_utils.write_sparse_file(te_labels, "data/amazonCat13/test_Y.txt")

In [13]:
def verify_warm_start_data(labels, rev_labels):
    assert labels.max()==1
    assert labels.min()==0
    assert rev_labels.max()==1
    assert rev_labels.min()==0
    
    unk_labels = labels - rev_labels
    assert unk_labels.min()==0 #if rev_labels is 1 at a loc, labels is certainly 1   
    print("if rev_labels is 1 at a position in matrix, labels is certainly 1 at this loc")
    
    print(f"avg. num labels per point is {labels.sum(axis=1).mean()}")
    print(f"avg. num revealed labels per point is {rev_labels.sum(axis=1).mean()}")
    print(f"avg. num hidden labels per point is {unk_labels.sum(axis=1).mean()}")
    
#     assert unk_labels.sum(axis=1)==0
    no_rel_label_ixs = np.argwhere(labels.sum(axis=1)==0)
    no_hidden_label_ixs = np.argwhere(labels.sum(axis=1)==0)
    np.testing.assert_almost_equal(no_rel_label_ixs, no_hidden_label_ixs)
    print("at least 1 label is hidden for each example")
    
    expected = unk_labels+rev_labels
    is_equal = (expected!=labels).nnz==0
    assert is_equal
#     np.testing.assert_almost_equal(expected.toarray(), labels.toarray())
    

# te_labels = data_utils.read_sparse_file("results/amazonCat13/test_Y.txt")
fracs = [20,40,60,80]
for frac in fracs:
    rev_labels = data_utils.read_sparse_file(f"{warm_start_data_path}/inc_test_Y_{frac}.txt")
    print(f"verifying for {frac}")
    verify_warm_start_data(te_labels, rev_labels)
    

verifying for 20
if rev_labels is 1 at a position in matrix, labels is certainly 1 at this loc
avg. num labels per point is 5.103092670440674
avg. num revealed labels per point is 0.585487425327301
avg. num hidden labels per point is 4.517605304718018
at least 1 label is hidden for each example
verifying for 40
if rev_labels is 1 at a position in matrix, labels is certainly 1 at this loc
avg. num labels per point is 5.103092670440674
avg. num revealed labels per point is 1.614302635192871
avg. num hidden labels per point is 3.4887900352478027
at least 1 label is hidden for each example
verifying for 60
if rev_labels is 1 at a position in matrix, labels is certainly 1 at this loc
avg. num labels per point is 5.103092670440674
avg. num revealed labels per point is 2.6793880462646484
avg. num hidden labels per point is 2.4237048625946045
at least 1 label is hidden for each example
verifying for 80
if rev_labels is 1 at a position in matrix, labels is certainly 1 at this loc
avg. num label

In [20]:
labels_nf = []
def format_label_embeddings(embeddings, label_tags_file):

    with open(label_tags_file, 'r', encoding='latin1') as f:
        temp = f.readlines()
    dataset_vocab = [item.rstrip("\n") for item in temp] #list of all words in given dataset
    del temp
#     dataset_vocab = dataset_vocab[:10000] #For Sample Toy
    print(len(dataset_vocab))

    label_embeddings = np.zeros((len(dataset_vocab), embeddings.vector_size))
    not_found_count = 0

    for i in range(len(dataset_vocab)):
        label_embeddings[i, :] = np.zeros(embeddings.vector_size)
        words = re.split(" |_|-", dataset_vocab[i])
        for word in words:
            try:
                label_embeddings[i, :] += embeddings[word]
            except KeyError:
                label_embeddings[i, :] += np.random.randn(embeddings.vector_size, )*0.01
                not_found_count+=1
                labels_nf.append(dataset_vocab[i])
    print("#Words with no word embeddings", not_found_count)
    return label_embeddings

def find(array, neq_val=None):
    """
    params:
    array: 2D array
    neq_val: 0 for user features, labels, and None for label/item features
    """
    vals = array[array!=neq_val]
    vals = np.array(vals).flatten()
    indeces = np.argwhere(array!=neq_val)+1
    indeces_ax0 = indeces[:, 0]
    indeces_ax1 = indeces[:, 1]
    
    return vals, indeces_ax0, indeces_ax1, indeces
    
def format_data(frac, save_path):
    tr_x = data_utils.read_sparse_file(f"{warm_start_data_path}/train_X.txt")
    tr_y = data_utils.read_sparse_file(f"{warm_start_data_path}/train_Y_{frac}.txt")
    te_x = data_utils.read_sparse_file(f"{warm_start_data_path}/test_X.txt")
    inc_te_y = data_utils.read_sparse_file(f"{warm_start_data_path}/inc_test_Y_{frac}.txt")
    exc_te_y = data_utils.read_sparse_file(f"{warm_start_data_path}/exc_test_Y_{frac}.txt")
    
    tr_x_v, tr_x_ix, tr_x_col_ix, _ = find(tr_x, 0)
    te_x_v, te_x_ix, te_x_col_ix, _ = find(te_x, 0)
    
    _, tr_y_ix, tr_y_col_ix, _ = find(tr_y, 0)
    _, inc_te_y_ix, inc_te_y_col_ix, _ = find(inc_te_y, 0)
    _, exc_te_y_ix, exc_te_y_col_ix, _ = find(exc_te_y, 0)
    
    
    np.savetxt(os.path.join(save_path, f"tr_x_v.csv"), tr_x_v, delimiter="\n")
    np.savetxt(os.path.join(save_path, f"tr_x_ix.csv"), tr_x_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"tr_x_col_ix.csv"), tr_x_col_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"te_x_v.csv"), te_x_v, delimiter="\n")
    np.savetxt(os.path.join(save_path, f"te_x_ix.csv"), te_x_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"te_x_col_ix.csv"), te_x_col_ix, delimiter="\n", fmt="%-d")
    
    np.savetxt(os.path.join(save_path, f"tr_y_ix_{frac}.csv"), tr_y_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"tr_y_col_ix_{frac}.csv"), tr_y_col_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"inc_{frac}_te_y_ix.csv"), inc_te_y_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"inc_{frac}_te_y_col_ix.csv"), inc_te_y_col_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"exc_{frac}_te_y_ix.csv"), exc_te_y_ix, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"exc_{frac}_te_y_col_ix.csv"), exc_te_y_col_ix, delimiter="\n", fmt="%-d")
    

In [21]:
word2vec500 = KeyedVectors.load_word2vec_format("../../word2vec/enwiki_20180420_500d.txt")

In [22]:
label_features = format_label_embeddings(word2vec500, f'{raw_data_path}/Yf.txt')

13330
#Words with no word embeddings 2873


In [26]:
label_features.shape

(13330, 500)

In [30]:
lf = csr_matrix(label_features)

In [31]:
data_utils.write_sparse_file(lf, "amzCat13_lf.txt")

In [1]:
os.getcwd()

NameError: name 'os' is not defined

In [44]:
fracs = [20,40,60,80]
for frac in fracs:
    format_data(frac, warm_start_data_path)

label_features = format_label_embeddings(word2vec500, f'{raw_data_path}/Yf.txt')

lf_v, lf_ix, lf_col_ix, _ = find(label_features, None)

np.savetxt(os.path.join(warm_start_data_path, f"w2v_emb_v.csv"), lf_v, delimiter="\n")
np.savetxt(os.path.join(warm_start_data_path, f"w2v_emb_ix.csv"), lf_ix, delimiter="\n", fmt="%-d")
np.savetxt(os.path.join(warm_start_data_path, f"w2v_emb_col_ix.csv"), lf_col_ix, delimiter="\n", fmt="%-d")

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/amaz.../Yf.txt'

In [20]:
frac=20

In [21]:
tr_x = data_utils.read_sparse_file(f"{warm_start_data_path}/train_X.txt")
tr_y = data_utils.read_sparse_file(f"{warm_start_data_path}/train_Y_{frac}.txt")
te_x = data_utils.read_sparse_file(f"{warm_start_data_path}/test_X.txt")
inc_te_y = data_utils.read_sparse_file(f"{warm_start_data_path}/inc_test_Y_{frac}.txt")
exc_te_y = data_utils.read_sparse_file(f"{warm_start_data_path}/exc_test_Y_{frac}.txt")
# label_features = format_label_embeddings(word2vec500, f'{raw_data_path}/Yf.txt')

tr_x_v, tr_x_ix, tr_x_col_ix, _ = find(tr_x, 0)
te_x_v, te_x_ix, te_x_col_ix, _ = find(te_x, 0)

_, tr_y_ix, tr_y_col_ix, _ = find(tr_y, 0)
_, inc_te_y_ix, inc_te_y_col_ix, _ = find(inc_te_y, 0)
_, exc_te_y_ix, exc_te_y_col_ix, _ = find(exc_te_y, 0)

In [22]:
tr_x.shape

(20762, 101938)

In [23]:
tr_y.shape

(20762, 30938)

In [24]:
te_x.shape

(6616, 101938)

In [25]:
inc_te_y.shape

(6616, 30938)

In [26]:
exc_te_y.shape

(6616, 30938)

In [27]:
tr_x_v.shape

(13890789,)

In [28]:
tr_x_ix.shape

(13890789,)

In [29]:
tr_x_col_ix.shape

(13890789,)

In [30]:
te_x_v.shape

(4364217,)

In [31]:
te_x_ix.shape

(4364217,)

In [32]:
te_x_col_ix.shape

(4364217,)

In [33]:
tr_y_ix.shape

(286242,)

In [34]:
tr_y_col_ix.shape

(286242,)

In [35]:
inc_te_y_ix.shape

(22537,)

In [36]:
inc_te_y_col_ix.shape

(22537,)

In [37]:
exc_te_y_ix.shape

(103335,)

In [38]:
exc_te_y_col_ix.shape

(103335,)

In [36]:
inc_te_y_ix[:10]

array([ 3,  3,  5, 10, 10, 10, 12, 12, 14, 15], dtype=int32)

In [2]:
check = np.loadtxt("warm_start_data/amazonCat13/tr_y_ix_20.csv")

In [3]:
check.shape

(6159056,)

In [55]:
frac

80

In [53]:
np.savetxt(f"exc_{frac}_te_y_ix.csv", exc_te_y_ix, delimiter="\n", fmt="%-d")

In [56]:
check2 = np.loadtxt("exc_80_te_y_ix.csv")

In [58]:
check2.shape

(427927,)